In [11]:
import os
import numpy as np

def read_annotations(file_path):
    with open(file_path, 'r') as file:
        data = np.loadtxt(file, usecols=(0,))
        return data

def calculate_timing_deviations(unperformed, performed):
    return performed - unperformed

def calculate_standard_deviation(deviations):
    return np.std(deviations)


def process_composer_folder(composer_folder):
    pieces = os.listdir(composer_folder)
    timing_deviations = []

    for piece in pieces:
        piece_folder = os.path.join(composer_folder, piece)
        if os.path.isdir(piece_folder):
            unperformed_path = os.path.join(piece_folder, 'midi_score_annotations.txt')
            if os.path.exists(unperformed_path):
                unperformed_annotations = read_annotations(unperformed_path)

                performed_files = [f for f in os.listdir(piece_folder) if f.endswith('_annotations.txt') and not f.startswith('midi_score')]
                for performed_file in performed_files:
                    performed_path = os.path.join(piece_folder, performed_file)
                    performed_annotations = read_annotations(performed_path)

                    deviations = calculate_timing_deviations(unperformed_annotations, performed_annotations)
                    timing_deviations.extend(deviations)

    return timing_deviations

def main():
    composers = os.listdir('Sample')

    composer_std_deviations = {}

    for composer in composers:
        composer_folder = os.path.join('Sample', composer)
        if os.path.isdir(composer_folder):
            timing_deviations = process_composer_folder(composer_folder)
            std_deviation = calculate_standard_deviation(timing_deviations)
            composer_std_deviations[composer] = std_deviation

    print("Standard Deviations:")
    for composer, std_deviation in composer_std_deviations.items():
        print(f"{composer}: {std_deviation}")

if __name__ == "__main__":
    main()




Standard Deviations:
Balakirev: 30.293283550249154
